<a href="https://colab.research.google.com/github/rahiakela/mlops-research-and-practice/blob/main/MLOps-Specialization/course-3-machine-learning-modeling-pipelines-in-production/week-3-high-performance-modeling/01_distributed_strategies_with_tf_and_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Distributed Strategies with TF and Keras